# 🧪 Resonance Protocol — Phase M2.5c: Fine-tuning Comparison

**Goal:** Compare model quality after fine-tuning on:
- Random subset (500 samples)
- SentenceTransformer-curated subset (500 samples)
- HDC-curated subset (500 samples)

**Success Criteria:**
- HDC-curated Loss ≤ ST-curated Loss → HDC is valid replacement
- HDC-curated Loss < Random Loss → curation works

---

## Step 1: Setup Environment

In [ ]:
# Install dependencies
!pip install -q transformers datasets peft accelerate bitsandbytes
!pip install -q sentence-transformers scikit-learn
!pip install -q tqdm numpy

In [ ]:
# Check GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Step 2: Load Dataset and Create Subsets

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
import random

# Load Alpaca dataset
print("Loading Alpaca dataset...")
dataset = load_dataset("tatsu-lab/alpaca", split="train")
print(f"Total samples: {len(dataset)}")

# Take subset for speed (2000 samples to curate from)
POOL_SIZE = 2000
SUBSET_SIZE = 500

random.seed(42)
pool_indices = random.sample(range(len(dataset)), POOL_SIZE)
pool = dataset.select(pool_indices)
print(f"Pool size: {len(pool)}")

In [ ]:
# Create text representations for encoding
def format_example(example):
    """Format instruction + input + output as single text"""
    text = f"Instruction: {example['instruction']}"
    if example.get('input'):
        text += f"\nInput: {example['input']}"
    text += f"\nOutput: {example['output']}"
    return text

pool_texts = [format_example(ex) for ex in pool]
print(f"Example text:\n{pool_texts[0][:500]}...")

In [ ]:
# Encode with SentenceTransformer
print("\nEncoding with SentenceTransformer...")
st_model = SentenceTransformer('all-MiniLM-L6-v2')
st_embeddings = st_model.encode(pool_texts, show_progress_bar=True)
print(f"ST embeddings shape: {st_embeddings.shape}")

In [ ]:
# HDC Encoder (Projection + Ternary)
class TernaryHDCEncoder:
    def __init__(self, input_dim=384, hd_dim=10000, sparsity=0.7, seed=42):
        self.input_dim = input_dim
        self.hd_dim = hd_dim
        self.sparsity = sparsity
        
        # Fixed random projection matrix
        np.random.seed(seed)
        self.projection = np.random.randn(input_dim, hd_dim).astype(np.float32)
        self.projection /= np.sqrt(input_dim)  # Normalize
    
    def encode(self, embeddings):
        """Project to HD space and ternarize"""
        # Project
        projected = embeddings @ self.projection
        
        # Ternarize: keep top/bottom (1-sparsity), zero middle
        ternary = np.zeros_like(projected)
        for i in range(len(projected)):
            vec = projected[i]
            threshold = np.percentile(np.abs(vec), self.sparsity * 100)
            ternary[i] = np.where(vec > threshold, 1,
                                   np.where(vec < -threshold, -1, 0))
        return ternary

# Encode with HDC
print("\nEncoding with HDC...")
hdc_encoder = TernaryHDCEncoder(input_dim=384, hd_dim=10000, sparsity=0.7)
hdc_embeddings = hdc_encoder.encode(st_embeddings)
print(f"HDC embeddings shape: {hdc_embeddings.shape}")
print(f"Sparsity: {(hdc_embeddings == 0).mean():.1%}")

In [ ]:
# Create 3 subsets

# 1. Random subset
random.seed(123)
random_indices = random.sample(range(POOL_SIZE), SUBSET_SIZE)
print(f"Random subset: {len(random_indices)} samples")

# 2. ST-curated (K-means clustering)
print("\nClustering ST embeddings...")
st_kmeans = KMeans(n_clusters=SUBSET_SIZE, random_state=42, n_init=10)
st_kmeans.fit(st_embeddings)

# Find nearest sample to each centroid
from sklearn.metrics import pairwise_distances
st_distances = pairwise_distances(st_kmeans.cluster_centers_, st_embeddings)
st_curated_indices = [int(np.argmin(st_distances[i])) for i in range(SUBSET_SIZE)]
st_curated_indices = list(set(st_curated_indices))  # Remove duplicates
print(f"ST-curated subset: {len(st_curated_indices)} samples")

# 3. HDC-curated (K-means clustering)
print("\nClustering HDC embeddings...")
hdc_kmeans = KMeans(n_clusters=SUBSET_SIZE, random_state=42, n_init=10)
hdc_kmeans.fit(hdc_embeddings)

hdc_distances = pairwise_distances(hdc_kmeans.cluster_centers_, hdc_embeddings)
hdc_curated_indices = [int(np.argmin(hdc_distances[i])) for i in range(SUBSET_SIZE)]
hdc_curated_indices = list(set(hdc_curated_indices))  # Remove duplicates
print(f"HDC-curated subset: {len(hdc_curated_indices)} samples")

In [ ]:
# Create datasets for fine-tuning
random_subset = pool.select(random_indices)
st_subset = pool.select(st_curated_indices)
hdc_subset = pool.select(hdc_curated_indices)

print(f"Random subset size: {len(random_subset)}")
print(f"ST-curated subset size: {len(st_subset)}")
print(f"HDC-curated subset size: {len(hdc_subset)}")

## Step 3: Fine-tuning Setup

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType

# Load model and tokenizer
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load in 4-bit for memory efficiency
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
print("Model loaded!")

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

print("LoRA config ready")

In [ ]:
# Tokenization function
def tokenize_function(examples):
    texts = []
    for i in range(len(examples['instruction'])):
        text = f"### Instruction:\n{examples['instruction'][i]}\n\n"
        if examples['input'][i]:
            text += f"### Input:\n{examples['input'][i]}\n\n"
        text += f"### Response:\n{examples['output'][i]}"
        texts.append(text)
    
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenize all subsets
print("Tokenizing datasets...")
random_tokenized = random_subset.map(tokenize_function, batched=True, remove_columns=random_subset.column_names)
st_tokenized = st_subset.map(tokenize_function, batched=True, remove_columns=st_subset.column_names)
hdc_tokenized = hdc_subset.map(tokenize_function, batched=True, remove_columns=hdc_subset.column_names)
print("Tokenization complete!")

## Step 4: Training Function

In [ ]:
def train_and_evaluate(train_dataset, run_name):
    """Train model and return loss history"""
    print(f"\n{'='*60}")
    print(f"Training: {run_name}")
    print(f"{'='*60}")
    
    # Create fresh PEFT model
    model = get_peft_model(base_model, lora_config)
    model.print_trainable_parameters()
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./results_{run_name}",
        num_train_epochs=3,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        warmup_steps=50,
        logging_steps=10,
        save_strategy="no",
        fp16=True,
        report_to="none",
        seed=42
    )
    
    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )
    
    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=data_collator
    )
    
    # Train
    train_result = trainer.train()
    
    # Get loss history
    loss_history = [log['loss'] for log in trainer.state.log_history if 'loss' in log]
    
    print(f"\nFinal loss: {loss_history[-1]:.4f}")
    
    # Clean up to free memory
    del model
    del trainer
    torch.cuda.empty_cache()
    
    return {
        'name': run_name,
        'final_loss': loss_history[-1],
        'loss_history': loss_history,
        'train_samples': len(train_dataset)
    }

## Step 5: Run Experiments

In [ ]:
# Train on all three subsets
results = {}

# 1. Random baseline
results['random'] = train_and_evaluate(random_tokenized, 'random')

# 2. ST-curated
results['st_curated'] = train_and_evaluate(st_tokenized, 'st_curated')

# 3. HDC-curated
results['hdc_curated'] = train_and_evaluate(hdc_tokenized, 'hdc_curated')

## Step 6: Results & Visualization

In [ ]:
import matplotlib.pyplot as plt

# Plot loss curves
plt.figure(figsize=(10, 6))

for name, data in results.items():
    plt.plot(data['loss_history'], label=f"{name} (final: {data['final_loss']:.4f})")

plt.xlabel('Training Steps (×10)')
plt.ylabel('Loss')
plt.title('Phase M2.5c: Fine-tuning Comparison\nRandom vs ST-Curated vs HDC-Curated')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('m2.5c_loss_curves.png', dpi=150)
plt.show()

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)

In [ ]:
# Summary table
print("\n📊 PHASE M2.5c RESULTS\n")
print(f"{'Method':<20} {'Samples':>10} {'Final Loss':>12} {'Status':>10}")
print("-" * 55)

random_loss = results['random']['final_loss']
st_loss = results['st_curated']['final_loss']
hdc_loss = results['hdc_curated']['final_loss']

# Determine winner
losses = {'Random': random_loss, 'ST-Curated': st_loss, 'HDC-Curated': hdc_loss}
winner = min(losses, key=losses.get)

for name, data in results.items():
    status = "👑 BEST" if data['final_loss'] == min(random_loss, st_loss, hdc_loss) else ""
    print(f"{name:<20} {data['train_samples']:>10} {data['final_loss']:>12.4f} {status:>10}")

print("\n" + "="*55)

# Analysis
print("\n🔬 ANALYSIS\n")

hdc_vs_random = ((random_loss - hdc_loss) / random_loss) * 100
hdc_vs_st = ((st_loss - hdc_loss) / st_loss) * 100
st_vs_random = ((random_loss - st_loss) / random_loss) * 100

print(f"HDC vs Random: {hdc_vs_random:+.2f}% {'✅ HDC better' if hdc_vs_random > 0 else '❌ Random better'}")
print(f"HDC vs ST:     {hdc_vs_st:+.2f}% {'✅ HDC better' if hdc_vs_st > 0 else '⚠️ ST better'}")
print(f"ST vs Random:  {st_vs_random:+.2f}% {'✅ ST better' if st_vs_random > 0 else '❌ Random better'}")

# Verdict
print("\n" + "="*55)
print("\n📋 VERDICT\n")

if hdc_loss <= st_loss and hdc_loss < random_loss:
    print("✅ SUCCESS: HDC-curated ≤ ST-curated < Random")
    print("   HDC is a valid replacement for ST in data curation!")
    verdict = "SUCCESS"
elif hdc_loss < random_loss:
    print("⚠️ PARTIAL SUCCESS: HDC-curated < Random, but ST is better")
    print("   HDC curation works, but doesn't beat ST embeddings.")
    verdict = "PARTIAL"
else:
    print("❌ FAILURE: Random ≥ HDC-curated")
    print("   Curation didn't help in this experiment.")
    verdict = "FAILURE"

print(f"\nWinner: {winner}")

In [ ]:
# Save results as JSON for RESEARCH_LOG
import json

output = {
    "phase": "M2.5c",
    "experiment": "Fine-tuning Comparison",
    "model": MODEL_NAME,
    "pool_size": POOL_SIZE,
    "subset_size": SUBSET_SIZE,
    "results": {
        "random": {
            "final_loss": float(random_loss),
            "samples": results['random']['train_samples']
        },
        "st_curated": {
            "final_loss": float(st_loss),
            "samples": results['st_curated']['train_samples']
        },
        "hdc_curated": {
            "final_loss": float(hdc_loss),
            "samples": results['hdc_curated']['train_samples']
        }
    },
    "comparison": {
        "hdc_vs_random_pct": float(hdc_vs_random),
        "hdc_vs_st_pct": float(hdc_vs_st),
        "st_vs_random_pct": float(st_vs_random)
    },
    "winner": winner,
    "verdict": verdict
}

with open('phase_m2.5c_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print("\n📁 Results saved to phase_m2.5c_results.json")
print("\nCopy this JSON to your RESEARCH_LOG.md:")
print(json.dumps(output, indent=2))

## Step 7: Download Results

Download:
- `phase_m2.5c_results.json` — raw results
- `m2.5c_loss_curves.png` — visualization

In [ ]:
from google.colab import files

# Download results
files.download('phase_m2.5c_results.json')
files.download('m2.5c_loss_curves.png')